In [1]:
import random
import itertools
import copy
import math
import pickle

In [2]:
import dataclasses
from __future__ import annotations
from typing import List, Dict, Set, Tuple

In [3]:
from tqdm.notebook import tqdm

In [4]:
SIDES = 6
NUMS = 6

In [5]:
class Dice:
    def __init__(self, nums:int=6, num_sides:int=SIDES):
        self.nums = nums
        self.sides = list(range(1, num_sides+1))
        self.dice = list(range(self.nums))
        self.active = set(self.dice)
        
        self.rolls = {i: 0 for i in self.dice}
        self.faces = tuple([0 for _ in self.dice])
        
    def __str__(self):
        return (f"Active dice: {"|".join([f"{die}:{self.rolls[die]}" for die in self.active])}")
        
    def add(self, dice:Set[int]):
        self.active.update(dice)
        
    def remove(self, dice:Set[int]):
        self.active.difference_update(dice)
    
    def roll(self):
        rolls = random.choices(self.sides, k=len(self.active))
        self.rolls = {}
        for idx, die in enumerate(self.active):
            self.rolls[die] = rolls[idx]    
        self.faces = tuple(sorted(rolls))
        
    def set_rolls(self, rolls):
        self.active = set(range(len(rolls)))
        self.rolls = {}
        for idx, die in enumerate(rolls):
            self.rolls[idx] = die
        self.faces = tuple(sorted(rolls))

In [6]:
class Rule:
    def __init__(self, name:str, pattern:Tuple[int], pts:int):
        self.name = name
        self.pattern = tuple(sorted(pattern))
        self.pts = pts
        
    def __str__(self):
        return self.name
    
    def __eq__(self, other:Rule):
        if not isinstance(other, Rule):
            return False
        return (self.pattern == other.pattern) and (self.pts == other.pts)
        
    def score(self, faces:Tuple[int]):
        if faces == self.pattern:
            return self.pts
        return 0
    
    def get_match(self, active:Set[int], rolls:Dict[int, int]):
        for subset in itertools.combinations(active, len(self.pattern)):
            values = tuple(sorted([rolls[i] for i in subset]))
            if values == self.pattern:
                return set(subset)
    
    def get_all_matches(self, active:Set[int], rolls:Dict[int, int]):
        matches = []
        for subset in itertools.combinations(active, len(self.pattern)):
            values = tuple(sorted([rolls[i] for i in subset]))
            if values == self.pattern:
                matches.append(set(subset))
        return matches

In [7]:
class Movelet:
    def __init__(self, rule:Rule=None, score:int=0, active:Set[int]=None):
        self.rule = rule
        self.score = score
        if active is None:
            self.active = set()
        else:
            self.active = active
            
    def __str__(self):
        return (f"{str(self.rule)} on dice: {", ".join([str(die) for die in self.active])} ({self.score})")
    
    def __hash__(self):
        rule_hash = self.rule.name if isinstance(self.rule, Rule) else None
        return hash((rule_hash, frozenset(self.active)))

    def __eq__(self, other):
        if not isinstance(other, Movelet):
            return False
        if self.rule is None:
            return other.rule is None
        if not isinstance(other.rule, Rule):
            return False
        return ((self.rule.name == other.rule.name) and (self.active == other.active))
    
    def apply(self, dice:Dice):
        self.active = self.rule.get_match(dice.active, dice.rolls)       

In [8]:
class Move:
    def __init__(self, movelets:Set[Movelet]=None, score:int=None, active:Set[int]=None):
        self.movelets = movelets
        if self.movelets is None:
            self.movelets = set()
        self.score = score
        if self.score is None:
            self.update_score()
        self.active = active
        if self.active is None:
            self.update_active()
            
    def __str__(self):
        return f"Move ({self.score}): " + "\n\t".join([str(movelet) for movelet in self.movelets])
    
    def __hash__(self):
        hashes = [hash(movelet) for movelet in self.movelets]
        return hash(frozenset(hashes))
    
    def __eq__(self, other):
        if not isinstance(other, Move):
            return False
        return self.movelets == other.movelets
        
    def update_score(self):
        self.score = 0
        for movelet in self.movelets:
            self.score += movelet.score
            
    def add_to_active(self, dice:Set[int]):
        if self.active & dice:
            raise ValueError("Overlapping dice in movelets")
        self.active |= dice
        
    def update_active(self):
        self.active = set()
        for movelet in self.movelets:
            self.add_to_active(movelet.active)
                
    def append(self, movelet:Movelet):
        self.movelets.add(movelet)            
        self.score += movelet.score
        self.add_to_active(movelet.active)
    
    def extend(self, move:Move):
        self.movelets.update(move.movelets)
        self.score += move.score
        self.add_to_active(move.active)
            
    def copy(self):
        return Move(movelets=copy.copy(self.movelets), score=self.score, active=copy.copy(self.active))
    
    def apply(self, dice:Dice):
        self.active = set()
        movelets = sorted(self.movelets, key=lambda movelet: (len(movelet.rule.pattern), movelet.score), reverse=True)
        for movelet in movelets:
            movelet.apply(dice)
            dice.active.difference_update(movelet.active)
            self.active.update(movelet.active)           

In [9]:
class RuleSet:
    def __init__(self, rules:List[Rule]):
        self.rules = rules
        self.rules.sort(key=lambda rule: (len(rule.pattern), rule.pts), reverse=True)
        
    def _get_all_moves_dfs(self, start_rule_idx:int, curr_move:Move, rem_active:Set[int], rolls:Dict[int, int]):
        all_moves = set()
        all_moves.add(curr_move)
        for ridx in range(start_rule_idx, len(self.rules)):
            rule = self.rules[ridx]
            for matches in rule.get_all_matches(rem_active, rolls):
                movelet = Movelet(rule=rule, score=rule.pts, active=matches)
                move = curr_move.copy()
                move.append(movelet)
                all_moves.update(self._get_all_moves_dfs(ridx, move, rem_active.difference(matches), rolls))
        return all_moves         
        
    def get_all_moves(self, dice:Dice):
        all_moves = self._get_all_moves_dfs(0, Move(), dice.active, dice.rolls)
        all_moves.remove(Move())
        return all_moves
    
    def get_best_moves(self, dice:Dice):
        best_moves = {}
        for move in self.get_all_moves(dice):
            key = len(move.active)
            if key not in best_moves or best_moves[key].score < move.score:
                best_moves[key] = move
        return best_moves
    
    def __eq__(self, other:RuleSet):
        if not isinstance(other, RuleSet):
            return False
        if len(self.rules) != len(other.rules):
            return False
        self_rules = sorted(self.rules, key=lambda rule: (rule.pattern, rule.pts), reverse=True)
        other_rules = sorted(other.rules, key=lambda rule: (rule.pattern, rule.pts), reverse=True)
        for idx in range(len(self_rules)):
            if self_rules[idx] != other_rules[idx]:
                return False
        return True

In [10]:
rules = [
    Rule("Single 1", [1], 100),
    Rule("Single 5", [5], 50),
    Rule("Three 1s", [1, 1, 1], 1000),
    Rule("Three 2s", [2, 2, 2],  200),
    Rule("Three 3s", [3, 3, 3],  300),
    Rule("Three 4s", [4, 4, 4],  400),
    Rule("Three 5s", [5, 5, 5],  500),
    Rule("Three 6s", [6, 6, 6],  600),
    Rule("Four 1s", [1, 1, 1, 1], 2000),
    Rule("Four 2s", [2, 2, 2, 2], 400),
    Rule("Four 3s", [3, 3, 3, 3], 600),
    Rule("Four 4s", [4, 4, 4, 4], 800),
    Rule("Four 5s", [5, 5, 5, 5], 1000),
    Rule("Four 6s", [6, 6, 6, 6], 1200),
    Rule("Five 1s", [1, 1, 1, 1, 1], 3000),
    Rule("Five 2s", [2, 2, 2, 2, 2], 600),
    Rule("Five 3s", [3, 3, 3, 3, 3], 900),
    Rule("Five 4s", [4, 4, 4, 4, 4], 1200),
    Rule("Five 5s", [5, 5, 5, 5, 5], 1500),
    Rule("Five 6s", [6, 6, 6, 6, 6], 1800),
    Rule("Six 1s", [1, 1, 1, 1, 1, 1], 4000),
    Rule("Six 2s", [2, 2, 2, 2, 2, 2],  800),
    Rule("Six 3s", [3, 3, 3, 3, 3, 3],  1200),
    Rule("Six 4s", [4, 4, 4, 4, 4, 4],  1600),
    Rule("Six 5s", [5, 5, 5, 5, 5, 5],  2000),
    Rule("Six 6s", [6, 6, 6, 6, 6, 6],  2400),
    Rule("1 to 6", [1, 2, 3, 4, 5, 6], 1500),
    Rule("1 to 5", [1, 2, 3, 4, 5], 1000),
    Rule("2 to 6", [2, 3, 4, 5, 6], 1000),
]

ruleset = RuleSet(rules)

In [11]:
@dataclasses.dataclass
class MoveData:
    rolls: Dict[int, int]=None
    move: Move=None
    passed:bool=False
    busted:bool=False
    reset:bool=False
        
    def __str__(self):
        out_str = ""
        if self.rolls != None:
            out_str += f"\nRolls: {"|".join([f"{die}:{self.rolls[die]}" for die in self.rolls])}" + "\n"
        if self.move != None:
            out_str += str(self.move) + "\n"
        if self.passed:
            out_str += "Pass!"
        elif self.busted:
            out_str += "Bust!"
        elif self.reset:
            out_str += "Reset!"
        return out_str

In [12]:
@dataclasses.dataclass
class RoundData:
    round_num: int=0
    player:Player=None
    moves:List[MoveData]=None
    old_total_pts:int=0
    new_total_pts:int=0
        
    def __str__(self):
        out_str = f"Round: {self.round_num}, Player: {self.player.name}\n\t"
        for move_data in self.moves:
            out_str += (str(move_data).replace("\n", "\n\t"))
        out_str += f"\n\tCurrent Round: {self.new_total_pts - self.old_total_pts}, Total Points: {self.new_total_pts}"
        return out_str

In [13]:
@dataclasses.dataclass
class GameData:
    rounds: List[RoundData]=None
    winner: Player=None
        
    def __str__(self):
        win_str = f"\nWinner: {self.winner.name}!"
        return "\n".join([str(round_data) for round_data in self.rounds]) + win_str

In [14]:
class Strategy:
    def should_roll(self, my_dice:Dice, my_total_pts:int, my_curr_pts:int, opp_total_pts:int, 
                            victory_pts:int, ruleset:Ruleset):
        return True
    
    def make_move(self, my_dice:Dice, my_total_pts:int, my_curr_pts:int, opp_total_pts:int, 
                            victory_pts:int, ruleset:Ruleset):
        return None
    
    def save(self, filename):
        with open(filename, "wb") as f:
            pickle.dump(self, f)

    @classmethod
    def load(cls, filename):
        with open(filename, "rb") as f:
            obj = pickle.load(f)
        return obj

In [15]:
class SimpleStrategy(Strategy):
    def __init__(self):
        self.victory_pts = None
        self.ruleset = None
        
        self.step = 1
        self.num_steps = float("inf")
        self.move_profiles = set()
        
        self.roll_dict = {}
        self.move_dict = {}
        
    def get_move_profile(self, dice):
        best_move = self.ruleset.get_best_moves(dice)
        num_active = len(dice.active)
        moves = [(idx, best_move[idx].score) for idx in best_move]
        return tuple([num_active, tuple(sorted(moves))])
        
        
    def init_move_profiles(self):
        self.move_profiles = set()
        total = sum([SIDES**num for num in range(1, NUMS+1)])
        with tqdm(total=total) as pbar: 
            for num in range(1, NUMS+1):
                dice = Dice(nums=num)
                for faces in itertools.product(*[range(1, SIDES+1) for _ in range(num)]):
                    faces = tuple(sorted(faces))
                    dice.set_rolls(faces)
                    move_profile = self.get_move_profile(dice)
                    self.move_profiles.add(move_profile)
                    pbar.update(1)                    
                    
    def get_num_steps(self):
        num_steps = 0
        for my_total_pts in range(self.victory_pts-self.step, -1, -self.step):
            for my_curr_pts in range(self.victory_pts-self.step-my_total_pts, -1, -self.step):
                for opp_total_pts in range(self.victory_pts-self.step, -1, -self.step):
                    num_steps += 1
        return num_steps    
        
    def init_roll_dict(self):
        self.roll_dict = {}
        total = self.num_steps*NUMS
        with tqdm(total=total) as pbar: 
            for my_total_pts in range(self.victory_pts-self.step, -1, -self.step):
                for my_curr_pts in range(self.victory_pts-self.step-my_total_pts, -1, -self.step):
                    for opp_total_pts in range(self.victory_pts-self.step, -1, -self.step):
                        for num in range(1, NUMS+1):
                            key = (my_total_pts, my_curr_pts, opp_total_pts, num)
                            self.roll_dict[key] = ((my_total_pts + my_curr_pts < self.victory_pts) and (num >= 3))
                            pbar.update(1)
                            
    def init_move_dict(self):
        self.move_dict = {}
        total = self.num_steps*len(self.move_profiles)
        with tqdm(total=total) as pbar: 
            for my_total_pts in range(self.victory_pts-self.step, -1, -self.step):
                for my_curr_pts in range(self.victory_pts-self.step-my_total_pts, -1, -self.step):
                    for opp_total_pts in range(self.victory_pts-self.step, -1, -self.step):
                        for move_profile in self.move_profiles:
                            key = (my_total_pts, my_curr_pts, opp_total_pts, move_profile)
                            if len(move_profile[1]) == 0:
                                self.move_dict[key] = None
                            else:
                                self.move_dict[key] = max(move_profile[1], key=lambda item: item[1])[0]
                            pbar.update(1)
        
        
    def init(self, victory_pts:int, ruleset:Ruleset):
        if victory_pts == self.victory_pts and ruleset == self.ruleset:
            return True
        
        self.victory_pts = victory_pts
        self.ruleset = ruleset
        
        self.step = math.gcd(*[rule.pts for rule in ruleset.rules])
        self.num_steps = self.get_num_steps()
        print("Initializing move_profiles...")
        self.init_move_profiles()
        print("Initializing roll_dict...")
        self.init_roll_dict()
        print("Initializing move_dict...")
        self.init_move_dict()
        return False
        
    def pickle(self, file):
        with open(file, "wb") as f:
            pickle.dump([self.roll_dict, self.move_dict], f)
        
        
    def should_roll(self, my_dice:Dice, my_total_pts:int, my_curr_pts:int, opp_total_pts:int, 
                            victory_pts:int, ruleset:Ruleset):
        self.init(victory_pts, ruleset)
        if my_total_pts + my_curr_pts >= victory_pts:
            return False
        key = (my_total_pts, my_curr_pts, opp_total_pts, len(my_dice.active))
        return self.roll_dict[key]
    
    def make_move(self, my_dice:Dice, my_total_pts:int, my_curr_pts:int, opp_total_pts:int, 
                            victory_pts:int, ruleset:Ruleset):
        self.init(victory_pts, ruleset)
        key = (my_total_pts, my_curr_pts, opp_total_pts, self.get_move_profile(my_dice))
        move = self.move_dict[key]
        if move is None:
            return None
        best_moves = ruleset.get_best_moves(my_dice)
        return best_moves[move]

In [16]:
class HumanStrategy(Strategy): 
    def __init__(self):
        self.rules = None
        
    def update_rules(self, ruleset:Ruleset):
        if self.rules is not None:
            return
        self.rules = {rule.name: rule for rule in ruleset.rules}

    def should_roll(self, my_dice:Dice, my_total_pts:int, my_curr_pts:int, opp_total_pts:int, 
                            victory_pts:int, ruleset:Ruleset):
        print(f"Your total points: {my_total_pts} | "
              f"Your current points: {my_curr_pts} | "
              f"Opponent's total points: {opp_total_pts}")
        answer = input(f"You have {len(my_dice.active)} dice to roll - "
                       f"select if you wish to roll (Y/N): ")
        while answer.lower().strip() not in ["y", "n"]:
            answer = input(f"You must choose between (Y/N): ")
        return answer.lower().strip() == "y"


    def _display_roll(self, my_dice: Dice):
        roll_str = "|".join(f"{die}:{my_dice.rolls[die]}" for die in my_dice.rolls)
        print(f"You rolled: {roll_str}")

    def _get_all_rule_names(self, all_valid_moves):
        return list(set([movelet.rule.name for move in all_valid_moves for movelet in move.movelets]))

    def _prompt_for_rule(self, all_valid_rule_names, move):
        rule_name_str = "Select the name of the rule you wish to apply: "
        if len(move.movelets) >= 1:
            all_valid_rule_names = all_valid_rule_names + ["Quit"]
            rule_name_str = "Select the name of the rule you wish to apply (or 'Quit' if you are done): "

        rule_name = input(rule_name_str).strip()
        while rule_name not in all_valid_rule_names:
            print("The rule you entered is not in the ruleset, these are the valid rules: ")
            print("\t" + ", ".join(all_valid_rule_names))
            rule_name = input(rule_name_str).strip()

        return None if rule_name == "Quit" else self.rules[rule_name]

    def _prompt_for_dice(self, my_dice:Dice, rule):
        while True:
            dice_str = input("Select the dice you wish to apply this rule on: ")
            try:
                dice = {int(die.strip()) for die in dice_str.split(",")}
            except ValueError:
                print("Enter them in the correct format - die indices separated by commas")
                continue

            if not (dice <= my_dice.active):
                print(f"You must only select from among the active dice: "
                      f"{', '.join(str(d) for d in my_dice.active)}")
                continue

            faces = tuple(sorted(my_dice.rolls[die] for die in dice))
            if rule.score(faces) == 0:
                print(f"Rule {rule.name} is not applicable on the selected dice")
                continue

            return dice

    def make_move(self, my_dice:Dice, my_total_pts:int, my_curr_pts:int, opp_total_pts:int, 
                            victory_pts:int, ruleset:Ruleset):
        self.update_rules(ruleset)
        self._display_roll(my_dice)
        
        move = Move()
        dice = copy.deepcopy(my_dice)
        while True:
            all_valid_moves = ruleset.get_all_moves(dice)
            if len(move.movelets) == 0 and not all_valid_moves:
                return
            elif not all_valid_moves:
                break
            all_valid_rule_names = self._get_all_rule_names(all_valid_moves)
        
            rule = self._prompt_for_rule(all_valid_rule_names, move)
            if rule is None:
                break

            new_dice = self._prompt_for_dice(dice, rule)
            movelet = Movelet(rule, rule.pts, new_dice)
            dice.active.difference_update(new_dice)
            move.append(movelet)

        return move

In [17]:
class OptimalStrategy(SimpleStrategy):
    def __init__(self, threshold=1e-3):
        super().__init__()
        self.threshold = threshold
        
        self.joint_dists = {}
        self.win_dp = {}
        self.joint_win_dp = {}
        
    def init_move_profiles_and_joint_dists(self):
        self.move_profiles = set()
        self.joint_dists = {}
        total = sum([SIDES**num for num in range(1, NUMS+1)])
        with tqdm(total=total) as pbar: 
            for num in range(1, NUMS+1):
                event_prob = (1/SIDES)**num
                dice = Dice(nums=num)
                for faces in itertools.product(*[range(1, SIDES+1) for _ in range(num)]):
                    faces = tuple(sorted(faces))
                    dice.set_rolls(faces)
                    move_profile = self.get_move_profile(dice)
                    self.move_profiles.add(move_profile)
                    if move_profile not in self.joint_dists:
                        self.joint_dists[move_profile] = 0
                    self.joint_dists[move_profile] += event_prob
                    pbar.update(1)
                    
    def init_win_dp(self):
        self.win_dp = {}
        total = self.num_steps*len(self.move_profiles)
        with tqdm(total=total) as pbar: 
            for my_total_pts in range(self.victory_pts-self.step, -1, -self.step):
                for my_curr_pts in range(self.victory_pts-self.step-my_total_pts, -1, -self.step):
                    for opp_total_pts in range(self.victory_pts-self.step, -1, -self.step):
                        for move_profile in self.move_profiles:
                            key = (my_total_pts, my_curr_pts, opp_total_pts, move_profile)
                            self.win_dp[key] = 0.5
                            pbar.update(1)
                    
    def init_joint_win_dp(self):
        self.joint_win_dp = {}
        total = self.num_steps*NUMS
        with tqdm(total=total) as pbar: 
            for my_total_pts in range(self.victory_pts-self.step, -1, -self.step):
                for my_curr_pts in range(self.victory_pts-self.step-my_total_pts, -1, -self.step):
                    for opp_total_pts in range(self.victory_pts-self.step, -1, -self.step):
                        for num in range(1, NUMS+1):
                            key = (my_total_pts, my_curr_pts, opp_total_pts, num)
                            self.joint_win_dp[key] = 0.5
                            pbar.update(1)
                            
    def get_win_dp_val(self, key:Tuple):
        my_total_pts, my_curr_pts, opp_total_pts, move_profile = key
        active_num, moves = move_profile
        if len(moves) == 0:
            opp_key = (opp_total_pts, 0, my_total_pts, NUMS)
            return 1 - self.get_joint_win_dp_val(opp_key)
        best_val = 0
        for num, pts in moves:
            rem_num = active_num - num
            if rem_num == 0:
                rem_num = NUMS
            move_key = (my_total_pts, my_curr_pts+pts, opp_total_pts, rem_num)
            move_val = self.get_joint_win_dp_val(move_key)
            if move_val > best_val:
                best_val = move_val
        return best_val
    
    def get_joint_win_dp_val(self, key:Tuple):
        my_total_pts, my_curr_pts, opp_total_pts, num = key
        if my_total_pts + my_curr_pts >= self.victory_pts:
            return 1.0
        return self.joint_win_dp[key]
    
    def train_win_dp(self):
        max_delta = 0
        with tqdm(total=len(self.win_dp)) as pbar: 
            for key in self.win_dp:
                old_val = self.win_dp[key]
                self.win_dp[key] = self.get_win_dp_val(key)
                new_val = self.win_dp[key]
                max_delta = max(max_delta, abs(new_val - old_val))
                pbar.update(1)
        return max_delta
    
    def get_roll_val_dict(self):
        roll_val_dict = {}
        total = len(self.win_dp) + len(self.joint_win_dp)
        with tqdm(total=total) as pbar:
            for key in self.joint_win_dp:
                roll_val_dict[key] = 0
                pbar.update(1)
            for key in self.win_dp:
                my_total_pts, my_curr_pts, opp_total_pts, move_profile = key
                joint_key = (my_total_pts, my_curr_pts, opp_total_pts, move_profile[0])
                prob = self.joint_dists[move_profile]
                roll_val_dict[joint_key] += prob*self.win_dp[key]
                pbar.update(1)
        return roll_val_dict
                
    def train_joint_win_dp(self):
        print("\tComputing roll_val_dict...")
        roll_val_dict = self.get_roll_val_dict()
        print("\tComputing pass_val_dict...")
        with tqdm(total=len(self.joint_win_dp)) as pbar:
            for key in self.joint_win_dp:
                my_total_pts, my_curr_pts, opp_total_pts, num = key
                if my_total_pts + my_curr_pts >= self.victory_pts:
                    self.joint_win_dp[key] = 1.0
                elif opp_total_pts >= self.victory_pts:
                    self.joint_win_dp[key] = 0.0
                else:
                    opp_key = (opp_total_pts, 0, my_total_pts+my_curr_pts, NUMS)
                    pass_val = 1 - self.get_joint_win_dp_val(opp_key)
                    self.joint_win_dp[key] = max(roll_val_dict[key], pass_val)
                pbar.update(1)
                
    def train(self, threshold=None):
        if threshold != None:
            self.threshold = threshold
        idx = 0
        while True:
            idx += 1
            print("*"*24)
            print(f"Iteration: {idx}...")
            print("Training joint_win_dp...")
            self.train_joint_win_dp()
            print("Training win_dp...")
            max_delta = self.train_win_dp()
            print(f"Maximum delta: {max_delta}")
            if max_delta < self.threshold:
                break
        
    def update_roll_dict(self):
        self.roll_dict = {}
        
        print("\tComputing roll_val_dict...")
        roll_val_dict = self.get_roll_val_dict()
        print("\tComputing roll_dict...")
        with tqdm(total=len(self.joint_win_dp)) as pbar:
            for key in self.joint_win_dp:
                self.roll_dict[key] = math.isclose(roll_val_dict[key], self.joint_win_dp[key], abs_tol=self.threshold)
                pbar.update(1)
                
    def update_move_dict(self):
        self.move_dict = {}
        with tqdm(total=len(self.win_dp)) as pbar:
            for key in self.win_dp:
                my_total_pts, my_curr_pts, opp_total_pts, move_profile = key
                active_num, moves = move_profile
                if len(moves) == 0:
                    self.move_dict[key] = None
                    pbar.update(1)
                    continue
                best_move, best_pts = None, 0
                for num, pts in moves:
                    rem_num = active_num - num
                    if rem_num == 0:
                        rem_num = NUMS
                    move_key = (my_total_pts, my_curr_pts+pts, opp_total_pts, rem_num)
                    val = self.joint_win_dp.get(move_key, 1.0)
                    if best_move == None or val > best_pts:
                        best_pts = val
                        best_move = num
                self.move_dict[key] = best_move
                pbar.update(1)
        
        
    def init(self, victory_pts:int, ruleset:Ruleset):
        if victory_pts == self.victory_pts and ruleset == self.ruleset:
            return True
        
        self.victory_pts = victory_pts
        self.ruleset = ruleset
        
        self.step = math.gcd(*[rule.pts for rule in ruleset.rules])
        self.num_steps = self.get_num_steps()
        print("Initializing move_profiles and joint_dists...")
        self.init_move_profiles_and_joint_dists()
        print("Initializing win_dp...")
        self.init_win_dp()
        print("Initializing joint_win_dp...")
        self.init_joint_win_dp()
        
        print("Training...")
        self.train()
        print("Updating roll_dict...")
        self.update_roll_dict()
        print("Updating move_dict...")
        self.update_move_dict()
        return False

In [18]:
class Player:
    def __init__(self, name:str, strategy:Strategy=Strategy(), printer:bool=False):
        self.name = name
        self.dice = Dice(nums=NUMS)
        self.strategy = strategy
        self.total_pts = 0
        self.curr_pts = 0
        self.printer = printer
        
    def reset(self):
        self.dice.active = set(self.dice.dice)
        self.total_pts = 0
        self.curr_pts = 0
        
    def play(self, opp, victory_pts, ruleset):
        key = (self.dice, self.total_pts, self.curr_pts, opp.total_pts, victory_pts, ruleset)
        move_data = MoveData()
        if not self.strategy.should_roll(*key):
            move_data.passed = True
            return move_data
        self.dice.roll()
        move_data.rolls = copy.copy(self.dice.rolls)
        move_data.move = self.strategy.make_move(*key)
        if move_data.move == None:
            move_data.busted = True
            return move_data
        self.curr_pts += move_data.move.score
        self.dice.active.difference_update(move_data.move.active)
        if self.dice.active == set():
            self.dice.active = set(self.dice.dice)
            move_data.reset = True
        return move_data

In [19]:
class Farkle:
    def __init__(self, player1:Player, player2:Player, victory_pts:int, ruleset:Ruleset=ruleset, printer:bool=False):
        self.players = [player1, player2]
        self.victory_pts = victory_pts
        self.ruleset = ruleset
        self.printer = printer
        
    def play_round(self, round_num, player, opp):
        round_data = RoundData(round_num=round_num, player=player, moves=[], old_total_pts=player.total_pts)
        while True:
            move_data = player.play(opp, self.victory_pts, self.ruleset)
            round_data.moves.append(move_data)
            if move_data.passed:
                player.total_pts += player.curr_pts
            if move_data.passed or move_data.busted:
                player.curr_pts = 0
                player.dice.active = set(player.dice.dice)
                break
        round_data.new_total_pts=player.total_pts
        if player.printer and self.printer:
            print(round_data)
        return round_data
    
    def play_game(self):
        for player in self.players:
            player.reset()
        game_data = GameData(rounds=[])
        round_num = 1
        player_idx = 0
        while True:
            player, opp = self.players[player_idx], self.players[1 - player_idx]
            round_data = self.play_round(round_num, player, opp)
            game_data.rounds.append(round_data)
            if player.total_pts >= self.victory_pts:
                game_data.winner = player
                if self.printer:
                    print(f"\nWinner: {game_data.winner.name}!")
                return game_data
            player_idx = 1 - player_idx
            round_num += 1

In [20]:
VICTORY_PTS = 2000

In [21]:
simple_strategy = SimpleStrategy()
simple_strategy.init(VICTORY_PTS, ruleset)
simple_strategy.save(f"simple_strategy({VICTORY_PTS}).pkl")

Initializing move_profiles...


  0%|          | 0/55986 [00:00<?, ?it/s]

Initializing roll_dict...


  0%|          | 0/196800 [00:00<?, ?it/s]

Initializing move_dict...


  0%|          | 0/7773600 [00:00<?, ?it/s]

In [22]:
player0 = Player("Me", strategy=HumanStrategy(), printer=False)
player1 = Player("Alice", strategy=simple_strategy, printer=True)
farkle = Farkle(player0, player1, victory_pts=VICTORY_PTS, printer=True)

In [23]:
winner = farkle.play_game().winner

Your total points: 0 | Your current points: 0 | Opponent's total points: 0
You have 6 dice to roll - select if you wish to roll (Y/N): Y
You rolled: 0:1|1:5|2:4|3:4|4:5|5:1
Select the name of the rule you wish to apply: Single 1
Select the dice you wish to apply this rule on: 0
Select the name of the rule you wish to apply (or 'Quit' if you are done): Quit
Your total points: 0 | Your current points: 100 | Opponent's total points: 0
You have 5 dice to roll - select if you wish to roll (Y/N): Y
You rolled: 1:6|2:1|3:3|4:2|5:4
Select the name of the rule you wish to apply: Single 1
Select the dice you wish to apply this rule on: 2
Your total points: 0 | Your current points: 200 | Opponent's total points: 0
You have 4 dice to roll - select if you wish to roll (Y/N): Y
You rolled: 1:4|3:4|4:1|5:4
Select the name of the rule you wish to apply: Single 1
Select the dice you wish to apply this rule on: 4
Select the name of the rule you wish to apply (or 'Quit' if you are done): Three 4s
Select 

In [24]:
optimal_strategy = OptimalStrategy(threshold=1e-7)
optimal_strategy.init(VICTORY_PTS, ruleset)
optimal_strategy.save(f"optimal_strategy({VICTORY_PTS}).pkl")

Initializing move_profiles and joint_dists...


  0%|          | 0/55986 [00:00<?, ?it/s]

Initializing win_dp...


  0%|          | 0/7773600 [00:00<?, ?it/s]

Initializing joint_win_dp...


...

  0%|          | 0/7970400 [00:00<?, ?it/s]

	Computing roll_dict...


  0%|          | 0/196800 [00:00<?, ?it/s]

Updating move_dict...


  0%|          | 0/7773600 [00:00<?, ?it/s]

In [25]:
player2 = Player("Bob", strategy=optimal_strategy, printer=True)
player3 = Player("Dave", strategy=SimpleStrategy.load(f"simple_strategy({VICTORY_PTS}).pkl"), printer=True)
player4 = Player("Carol", strategy=OptimalStrategy.load(f"optimal_strategy({VICTORY_PTS}).pkl"), printer=True)

In [27]:
farkle.players = [player1, player2]
farkle.play_game()

Round: 1, Player: Alice
	
	Rolls: 0:1|1:6|2:6|3:4|4:3|5:1
	Move (200): Single 1 on dice: 5 (100)
		Single 1 on dice: 0 (100)
	
	Rolls: 1:1|2:6|3:3|4:2
	Move (100): Single 1 on dice: 1 (100)
	
	Rolls: 2:2|3:1|4:1
	Move (200): Single 1 on dice: 4 (100)
		Single 1 on dice: 3 (100)
	Pass!
	Current Round: 500, Total Points: 500
Round: 2, Player: Bob
	
	Rolls: 0:5|1:1|2:6|3:1|4:3|5:2
	Move (100): Single 1 on dice: 3 (100)
	
	Rolls: 0:6|1:5|2:3|4:5|5:5
	Move (500): Three 5s on dice: 1, 4, 5 (500)
	Pass!
	Current Round: 600, Total Points: 600
Round: 3, Player: Alice
	
	Rolls: 0:6|1:5|2:2|3:5|4:2|5:4
	Move (100): Single 5 on dice: 3 (50)
		Single 5 on dice: 1 (50)
	
	Rolls: 0:5|2:3|4:4|5:4
	Move (50): Single 5 on dice: 0 (50)
	
	Rolls: 2:3|4:5|5:2
	Move (50): Single 5 on dice: 4 (50)
	Pass!
	Current Round: 200, Total Points: 700
Round: 4, Player: Bob
	
	Rolls: 0:3|1:3|2:1|3:1|4:4|5:6
	Move (100): Single 1 on dice: 3 (100)
	
	Rolls: 0:6|1:4|2:1|4:5|5:2
	Move (100): Single 1 on dice: 2 (100)
	
	R

GameData(rounds=[RoundData(round_num=1, player=<__main__.Player object at 0x00000151A6E69160>, moves=[MoveData(rolls={0: 1, 1: 6, 2: 6, 3: 4, 4: 3, 5: 1}, move=<__main__.Move object at 0x00000151A6D8CC50>, passed=False, busted=False, reset=False), MoveData(rolls={1: 1, 2: 6, 3: 3, 4: 2}, move=<__main__.Move object at 0x00000151A6D8E480>, passed=False, busted=False, reset=False), MoveData(rolls={2: 2, 3: 1, 4: 1}, move=<__main__.Move object at 0x00000151A6D8C770>, passed=False, busted=False, reset=False), MoveData(rolls=None, move=None, passed=True, busted=False, reset=False)], old_total_pts=0, new_total_pts=500), RoundData(round_num=2, player=<__main__.Player object at 0x00000151A6DB7560>, moves=[MoveData(rolls={0: 5, 1: 1, 2: 6, 3: 1, 4: 3, 5: 2}, move=<__main__.Move object at 0x00000151A6C58CB0>, passed=False, busted=False, reset=False), MoveData(rolls={0: 6, 1: 5, 2: 3, 4: 5, 5: 5}, move=<__main__.Move object at 0x00000151A6C599A0>, passed=False, busted=False, reset=False), MoveData

In [30]:
def play_batch_games(player1, player2, N=10**5):
    farkle.players = [player1, player2]
    farkle.printer = False
    wins = 0
    random.seed(42)
    with tqdm(total=N) as pbar:
        for i in range(N):
            if farkle.play_game().winner == farkle.players[0]:
                wins += 1
            pbar.update(1)

    print(wins/N)

In [31]:
play_batch_games(player1, player3) # simple vs simple

  0%|          | 0/100000 [00:00<?, ?it/s]

0.58145


In [32]:
play_batch_games(player1, player4) # simple vs optimal

  0%|          | 0/100000 [00:00<?, ?it/s]

0.4768


In [33]:
play_batch_games(player2, player3) # optimal vs simple

  0%|          | 0/100000 [00:00<?, ?it/s]

0.67804


In [34]:
play_batch_games(player2, player4) # optimal vs optimal

  0%|          | 0/100000 [00:00<?, ?it/s]

0.57818


In [36]:
farkle.players = [player0, player2]
farkle.printer = True
winner = farkle.play_game().winner

Your total points: 0 | Your current points: 0 | Opponent's total points: 0
You have 6 dice to roll - select if you wish to roll (Y/N): Y
You rolled: 0:4|1:1|2:2|3:1|4:5|5:2
Select the name of the rule you wish to apply: What
The rule you entered is not in the ruleset, these are the valid rules: 
	Single 5, Single 1
Select the name of the rule you wish to apply: Single 1
Select the dice you wish to apply this rule on: 1
Select the name of the rule you wish to apply (or 'Quit' if you are done): Quit
Your total points: 0 | Your current points: 100 | Opponent's total points: 0
You have 5 dice to roll - select if you wish to roll (Y/N): Y
You rolled: 0:4|2:5|3:2|4:1|5:5
Select the name of the rule you wish to apply: Single 1
Select the dice you wish to apply this rule on: 4
Select the name of the rule you wish to apply (or 'Quit' if you are done): Quit
Your total points: 0 | Your current points: 200 | Opponent's total points: 0
You have 4 dice to roll - select if you wish to roll (Y/N): Y
Y